In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
import mysql.connector as connector

In [2]:
#!pip install mysql-connector-python

In [3]:
stock = yf.Ticker("NVDA") 

In [4]:
stock_historical = stock.history(period="max", interval="1d")
stock_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22 00:00:00-05:00,0.040122,0.044779,0.035585,0.037615,2714688000,0.0,0.0
1999-01-25 00:00:00-05:00,0.040600,0.042033,0.037615,0.041556,510480000,0.0,0.0
1999-01-26 00:00:00-05:00,0.042033,0.042869,0.037734,0.038331,343200000,0.0,0.0
1999-01-27 00:00:00-05:00,0.038451,0.039406,0.036301,0.038212,244368000,0.0,0.0
1999-01-28 00:00:00-05:00,0.038212,0.038451,0.037854,0.038092,227520000,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-10 00:00:00-05:00,130.089996,135.000000,129.960007,133.570007,216989100,0.0,0.0
2025-02-11 00:00:00-05:00,132.580002,134.479996,131.020004,132.800003,178902400,0.0,0.0
2025-02-12 00:00:00-05:00,130.020004,132.240005,129.080002,131.139999,160278600,0.0,0.0


In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(stock_historical)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22 00:00:00-05:00,0.040122,0.044779,0.035585,0.037615,2714688000,0.0,0.0
1999-01-25 00:00:00-05:00,0.040600,0.042033,0.037615,0.041556,510480000,0.0,0.0
1999-01-26 00:00:00-05:00,0.042033,0.042869,0.037734,0.038331,343200000,0.0,0.0
1999-01-27 00:00:00-05:00,0.038451,0.039406,0.036301,0.038212,244368000,0.0,0.0
1999-01-28 00:00:00-05:00,0.038212,0.038451,0.037854,0.038092,227520000,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-10 00:00:00-05:00,130.089996,135.000000,129.960007,133.570007,216989100,0.0,0.0
2025-02-11 00:00:00-05:00,132.580002,134.479996,131.020004,132.800003,178902400,0.0,0.0
2025-02-12 00:00:00-05:00,130.020004,132.240005,129.080002,131.139999,160278600,0.0,0.0


In [7]:
try:
    conn = connector.connect(user='root', password='123456', host='localhost', port='3306', database='my_db')
except connector.Error as er:
    print('Error code:', er.errorno)
    print('Error msg:', er.msg)

In [8]:
mycursor = conn.cursor()

mycursor.execute("SHOW TABLES")

tables = [table[0] for table in mycursor.fetchall()]

if 'stock_price' not in tables:
    mycursor.execute("CREATE TABLE stock_price (Date DATE, Open FLOAT(24), High FLOAT(24), Low FLOAT(24), Close FLOAT(24), Volume FLOAT(24), Dividends FLOAT(24), Stock_Splits FLOAT(24))")
    print("Table 'stock_price' created successfully.")
else:
    print("Table 'stock_price' already exists.")


Table 'stock_price' already exists.


In [9]:
# mycursor.execute('SELECT * FROM stock_price')

# hist_price = mycursor.fetchall()


# mycursor.close()
# conn.close()

In [10]:
df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,6558.000000,6558.000000,6558.000000,6558.000000,6.558000e+03,6558.000000,6558.000000
mean,8.768930,8.929121,8.591945,8.768531,5.991103e+08,0.000028,0.003278
std,23.925027,24.351994,23.421474,23.907205,4.307236e+08,0.000358,0.140903
min,0.032003,0.032600,0.030569,0.031286,1.968000e+07,0.000000,0.000000
25%,0.257527,0.264589,0.250689,0.257739,3.384780e+08,0.000000,0.000000
50%,0.437144,0.443609,0.430419,0.437176,5.002635e+08,0.000000,0.000000
75%,4.592201,4.686506,4.550566,4.597059,7.307002e+08,0.000000,0.000000
max,153.029999,153.130005,147.820007,149.429993,9.230856e+09,0.010000,10.000000


In [11]:
# df.set_index('Date', drop=True, inplace=True)


In [12]:
#sql ="INSERT INTO"

sql = "INSERT INTO stock_price (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

In [13]:
# Convert DataFrame to list of tuples without the index
tuples_list = list(df.itertuples(index=True, name=None))


In [14]:
#need to use executemany to insert multiple rows
mycursor.executemany(sql, tuples_list)


In [15]:
#commit from the connection
conn.commit()

mycursor.close()
conn.close()